---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [10]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [11]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [12]:
def graph_identification():
    
    # Your Code Here

    res=list()
    res.append('PA')
    res.append('SW_L')
    res.append('SW_L')
    res.append('PA')
    res.append('SW_H')
    return res
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [13]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
def salary_predictions():
    
    # Your Code Here
    df = pd.DataFrame(index=G.nodes())
    df['departement'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['management'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['cc'] = pd.Series(nx.closeness_centrality(G))
    df['bc'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False))
    df['pr'] = pd.Series(nx.pagerank(G, alpha=0.8))
    df['hub'] = pd.Series(nx.hits(G)[0])
    df['autho'] = pd.Series(nx.hits(G)[1])
    
    X_train = df[~(df['management'].isnull())].drop('management', axis=1)
    y_train = df[~(df['management'].isnull())]['management']
    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)  
    
    to_predict = df[df['management'].isnull()].drop('management', axis=1)
    res = pd.Series(index=to_predict.index, data =rfc.predict_proba(to_predict)[:,1])
    #plusieurs paramètres de depth/pareil avec svm
    scores = cross_val_score(rfc, X_train, y_train, cv=5,
                        scoring=('roc_auc'))
    return  res
salary_predictions()

1       0.1
2       1.0
5       1.0
8       0.1
14      0.0
18      0.1
27      0.1
30      0.5
31      0.4
34      0.1
37      0.0
40      0.1
45      0.0
54      0.5
55      0.6
60      0.4
62      1.0
65      1.0
77      0.0
79      0.0
97      0.0
101     0.0
103     0.3
108     0.1
113     0.1
122     0.0
141     0.2
142     1.0
144     0.1
145     0.2
       ... 
913     0.0
914     0.0
915     0.0
918     0.1
923     0.0
926     0.0
931     0.0
934     0.0
939     0.0
944     0.0
945     0.0
947     0.2
950     0.1
951     0.0
953     0.0
959     0.0
962     0.0
963     0.4
968     0.0
969     0.0
974     0.0
984     0.0
987     0.0
989     0.0
991     0.0
992     0.0
994     0.0
996     0.0
1000    0.0
1001    0.0
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
def new_connections_predictions():    
    # Your Code Here
    df = pd.DataFrame(index=future_connections.index)
    df['future_connection'] = future_connections['Future Connection']
    df['index'] = df.index
    df['CN'] = df['index'].apply(lambda x: len(list(nx.common_neighbors(G, x[0],x[1]))))
    df['JC']=df['index'].apply(lambda x: list(nx.jaccard_coefficient(G,[(x[0],x[1])]))[0][2])
    df['RAI']=df['index'].apply(lambda x: list(nx.resource_allocation_index(G,[(x[0],x[1])]))[0][2])
    df['AAI']=df['index'].apply(lambda x: list(nx.adamic_adar_index(G,[(x[0],x[1])]))[0][2])
    df['PA']=df['index'].apply(lambda x: list(nx.preferential_attachment(G,[(x[0],x[1])]))[0][2])
    df=df.drop('index',axis=1)
    X_train = df[~(df['future_connection'].isnull())].drop('future_connection', axis=1)
    y_train = df[~(df['future_connection'].isnull())]['future_connection']
    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)  
    
    to_predict = df[df['future_connection'].isnull()].drop('future_connection', axis=1)
    res = pd.Series(index=to_predict.index, data =rfc.predict_proba(to_predict)[:,1])
    return res
new_connections_predictions()

(107, 348)    0.000000
(542, 751)    0.011899
(20, 426)     0.500000
(50, 989)     0.014124
(942, 986)    0.010798
(324, 857)    0.015352
(13, 710)     0.100000
(19, 271)     0.100000
(319, 878)    0.010239
(659, 707)    0.013724
(49, 843)     0.014431
(208, 893)    0.003822
(377, 469)    0.000000
(405, 999)    0.000000
(129, 740)    0.000000
(292, 618)    0.000000
(239, 689)    0.005697
(359, 373)    0.167371
(53, 523)     0.000000
(276, 984)    0.008238
(202, 997)    0.009134
(604, 619)    0.000000
(270, 911)    0.015168
(261, 481)    0.000000
(200, 450)    0.900000
(213, 634)    0.016951
(644, 735)    0.000000
(346, 553)    0.000000
(521, 738)    0.017189
(422, 953)    0.000000
                ...   
(672, 848)    0.015168
(28, 127)     1.000000
(202, 661)    0.010777
(54, 195)     1.000000
(295, 864)    0.011218
(814, 936)    0.010675
(839, 874)    0.010798
(139, 843)    0.012010
(461, 544)    0.008563
(68, 487)     0.000000
(622, 932)    0.014674
(504, 936)    0.000000
(479, 528) 